In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [2]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.models import resnet50, resnet101
import time

In [3]:
# --- 1. Définition du modèle SE-ResNet-50 (adapté pour 10 classes) ---
class SEBlock(nn.Module):
    def __init__(self, in_channels, r=16):
        super(SEBlock, self).__init__()
        self.squeeze = nn.AdaptiveAvgPool2d(1)
        self.excitation = nn.Sequential(
            nn.Linear(in_channels, in_channels // r),
            nn.ReLU(inplace=True),
            nn.Linear(in_channels // r, in_channels),
            nn.Sigmoid()
        )

    def forward(self, x):
        batch_size, num_channels, _, _ = x.size()
        y = self.squeeze(x).view(batch_size, num_channels)
        y = self.excitation(y).view(batch_size, num_channels, 1, 1)
        return x * y.expand_as(x)

class Bottleneck(nn.Module):
    expansion = 4

    def __init__(self, in_channels, out_channels, stride=1, downsample=None, r=16):
        super(Bottleneck, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, bias=False)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(out_channels)
        self.conv3 = nn.Conv2d(out_channels, out_channels * self.expansion, kernel_size=1, stride=1, bias=False)
        self.bn3 = nn.BatchNorm2d(out_channels * self.expansion)
        self.se_block = SEBlock(out_channels * self.expansion, r)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out = self.relu(out)
        out = self.conv3(out)
        out = self.bn3(out)
        out = self.se_block(out)

        if self.downsample is not None:
            residual = self.downsample(residual)
        
        out += residual
        out = self.relu(out)
        return out

class SEResNet(nn.Module):
    def __init__(self, block, layers, num_classes=10):
        super(SEResNet, self).__init__()
        self.in_channels = 64
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False) # Changed for CIFAR-10
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        
        self.layer1 = self._make_layer(block, 64, layers[0])
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)

        self.avgpool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(512 * block.expansion, num_classes)

    def _make_layer(self, block, out_channels, num_blocks, stride=1):
        downsample = None
        if stride != 1 or self.in_channels != out_channels * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.in_channels, out_channels * block.expansion, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(out_channels * block.expansion),
            )
        
        layers = []
        layers.append(block(self.in_channels, out_channels, stride, downsample))
        self.in_channels = out_channels * block.expansion
        for _ in range(1, num_blocks):
            layers.append(block(self.in_channels, out_channels))
        
        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        
        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        
        x = self.avgpool(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        
        return x


In [4]:
def se_resnet50(num_classes=100):
    return SEResNet(Bottleneck, [3, 4, 6, 3], num_classes)

In [5]:
# --- 2. Préparation des données CIFAR-10 ---
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])


In [6]:
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

100%|██████████| 169M/169M [00:03<00:00, 47.9MB/s] 


In [7]:
print(device)

cuda:0


In [8]:


# --- 3. Fonction d'entraînement et d'évaluation ---
def train_and_evaluate(model,model_name, num_epochs=100):
    # model_name = type(model).__name__
    # if model_name == "ResNet":
    #     model_name += "-50" if sum(model.layers) == 16 else "-101"

    model.to(device)
    criterion = nn.CrossEntropyLoss()
    optimizer = optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=5e-4)
    scheduler = optim.lr_scheduler.MultiStepLR(optimizer, milestones=[30, 60, 90], gamma=0.1)

    print(f"\n--- Début de l'entraînement pour {model_name} ---")
    start_time = time.time()
    for epoch in range(num_epochs):
        model.train()
        running_loss = 0.0
        for i, data in enumerate(trainloader, 0):
            inputs, labels = data[0].to(device), data[1].to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        
        scheduler.step()

        # Évaluation sur le jeu de test
        model.eval()
        correct = 0
        total = 0
        with torch.no_grad():
            for data in testloader:
                images, labels = data[0].to(device), data[1].to(device)
                outputs = model(images)
                _, predicted = torch.max(outputs.data, 1)
                total += labels.size(0)
                correct += (predicted == labels).sum().item()
        
        accuracy = 100 * correct / total
        print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {running_loss / len(trainloader):.4f}, Accuracy: {accuracy:.2f}%')

    end_time = time.time()
    training_time = end_time - start_time
    print(f"--- Entraînement terminé pour {model_name} en {training_time:.2f} secondes. ---")
    return accuracy, training_time

In [9]:


# --- 4. Exécution de la comparaison ---
results = {}

In [10]:

# Modèle 2: ResNet-50
resnet_50_model = resnet50(weights=None, num_classes=100) # <-- Changé pour 100
resnet_50_accuracy, resnet_50_time = train_and_evaluate(resnet_50_model,'ResNet-50')
results['ResNet-50'] = {'Accuracy': resnet_50_accuracy, 'Time': resnet_50_time}

# Modèle 3: ResNet-101
resnet_101_model = resnet101(weights=None, num_classes=100) # <-- Changé pour 100
resnet_101_accuracy, resnet_101_time = train_and_evaluate(resnet_101_model,'ResNet-101')
results['ResNet-101'] = {'Accuracy': resnet_101_accuracy, 'Time': resnet_101_time}




--- Début de l'entraînement pour ResNet-50 ---
Epoch 1/100, Loss: 5.6662, Accuracy: 4.10%
Epoch 2/100, Loss: 4.0678, Accuracy: 9.01%
Epoch 3/100, Loss: 3.7838, Accuracy: 13.28%
Epoch 4/100, Loss: 3.5328, Accuracy: 17.75%
Epoch 5/100, Loss: 3.3574, Accuracy: 19.81%
Epoch 6/100, Loss: 3.1973, Accuracy: 22.35%
Epoch 7/100, Loss: 3.0459, Accuracy: 24.12%
Epoch 8/100, Loss: 2.9115, Accuracy: 24.98%
Epoch 9/100, Loss: 2.7825, Accuracy: 27.56%
Epoch 10/100, Loss: 2.6699, Accuracy: 29.81%
Epoch 11/100, Loss: 2.5720, Accuracy: 32.45%
Epoch 12/100, Loss: 2.4723, Accuracy: 31.32%
Epoch 13/100, Loss: 2.3851, Accuracy: 32.66%
Epoch 14/100, Loss: 2.2884, Accuracy: 33.08%
Epoch 15/100, Loss: 2.2133, Accuracy: 33.46%
Epoch 16/100, Loss: 2.1261, Accuracy: 34.73%
Epoch 17/100, Loss: 2.0393, Accuracy: 36.89%
Epoch 18/100, Loss: 1.9539, Accuracy: 35.18%
Epoch 19/100, Loss: 1.8747, Accuracy: 36.11%
Epoch 20/100, Loss: 1.7993, Accuracy: 38.41%
Epoch 21/100, Loss: 1.7327, Accuracy: 34.37%
Epoch 22/100, Loss

In [11]:
# Modèle 1: SE-ResNet-50
se_resnet_50_model = se_resnet50()
se_accuracy, se_time = train_and_evaluate(se_resnet_50_model,"SE-ResNet-50")
results['SE-ResNet-50'] = {'Accuracy': se_accuracy, 'Time': se_time}



--- Début de l'entraînement pour SE-ResNet-50 ---
Epoch 1/100, Loss: 4.0652, Accuracy: 12.68%
Epoch 2/100, Loss: 3.3511, Accuracy: 23.09%
Epoch 3/100, Loss: 2.9446, Accuracy: 28.58%
Epoch 4/100, Loss: 2.5369, Accuracy: 33.59%
Epoch 5/100, Loss: 2.2186, Accuracy: 38.84%
Epoch 6/100, Loss: 1.9759, Accuracy: 43.69%
Epoch 7/100, Loss: 1.7801, Accuracy: 44.82%
Epoch 8/100, Loss: 1.6174, Accuracy: 46.56%
Epoch 9/100, Loss: 1.4871, Accuracy: 48.71%
Epoch 10/100, Loss: 1.3830, Accuracy: 48.51%
Epoch 11/100, Loss: 1.3014, Accuracy: 50.32%
Epoch 12/100, Loss: 1.2194, Accuracy: 48.58%
Epoch 13/100, Loss: 1.1679, Accuracy: 50.05%
Epoch 14/100, Loss: 1.1016, Accuracy: 50.57%
Epoch 15/100, Loss: 1.0471, Accuracy: 49.33%
Epoch 16/100, Loss: 1.0063, Accuracy: 50.49%
Epoch 17/100, Loss: 0.9515, Accuracy: 48.53%
Epoch 18/100, Loss: 0.9161, Accuracy: 51.18%
Epoch 19/100, Loss: 0.8931, Accuracy: 51.63%
Epoch 20/100, Loss: 0.8663, Accuracy: 48.39%
Epoch 21/100, Loss: 0.8478, Accuracy: 48.68%
Epoch 22/100,

In [12]:
# --- 5. Affichage des résultats ---
print("\n" + "="*50)
print("             RÉSUMÉ DE LA COMPARAISON")
print("="*50)
for model_name, data in results.items():
    print(f"Modèle: {model_name}")
    print(f"  Précision finale: {data['Accuracy']:.2f}%")
    print(f"  Temps d'entraînement: {data['Time']:.2f} secondes")
    print("-" * 30)



             RÉSUMÉ DE LA COMPARAISON
Modèle: ResNet-50
  Précision finale: 48.46%
  Temps d'entraînement: 2872.56 secondes
------------------------------
Modèle: ResNet-101
  Précision finale: 48.45%
  Temps d'entraînement: 4678.54 secondes
------------------------------
Modèle: SE-ResNet-50
  Précision finale: 63.93%
  Temps d'entraînement: 24268.87 secondes
------------------------------
